# Zgony według tygodni, GUS
## W podziale na województwa, powiaty i 5-letnie grupy wiekowe

Potrzebne funkcje w plikach:
* helper_functions.py
* govpl_function.py

Uwaga: 
...

## TODO
* ...

In [4]:
"""
Set PYTHONPATH for modules in parallel directory

https://stackoverflow.com/questions/3108285/in-python-script-how-do-i-set-pythonpath/3108301

"""

import sys

try:
    sys.path.index('..') # Or os.getcwd() for this directory
except ValueError:
    sys.path.append('..') # Or os.getcwd() for this directory

#--------------------------------------------------------------

from share.helper_functions import (
    getfile, 
    unzip, 
    xlsx2xls, 
    display_all
)
from typing import NamedTuple
from share.gus_functions import (
    GUSparams,
    Analysis
    )
# import govpl_functions as govpl
import pandas as pd
import importlib
import glob
import os

In [5]:
data_dir = './gus_data'
img_dir = './images'
url =\
'https://stat.gov.pl/download/gfx/portalinformacyjny/pl/defaultaktualnosci/5468/39/2/1/zgony_wg_tygodni.zip'
zipfile = 'zgony_wg_tygodni.zip'
zipfile_path = os.sep.join([data_dir,zipfile])
zip_dir = os.sep.join([data_dir,'zgony_wg_tygodni'])
file_prefix = 'Zgony wedêug tygodni w Polsce_'
file_prefix_terminal = 'Zgony\ wedêug\ tygodni\ w\ Polsce_'
file_suffix = '.xlsx'
libreoffice_cmd = 'libreoffice7.2'

mygus = Analysis()

mygus.params = GUSparams(
    data_dir,
    img_dir,
    url,
    zipfile,
    zipfile_path,
    zip_dir,
    file_prefix,
    file_prefix_terminal,
    file_suffix,
    libreoffice_cmd
)


In [6]:
mygus.download_unzip_convert_to_xls()

./gus_data/zgony_wg_tygodni.zip exists, so not downloaded
*.xlsx or *.xls files exist in ./gus_data/zgony_wg_tygodni, so zip file not extracted
*.xls files exist in ./gus_data/zgony_wg_tygodni, so *.xlsx files not converted to *.xls


In [7]:
# %load_ext autoreload
# %autoreload

# from share.helper_functions import (
#     getfile, 
#     unzip, 
#     xlsx2xls, 
#     display_all
# )

In [8]:
# year = 2021
# df = mygus.read_xls_year(year)
# df = mygus.format_df(df,year)

In [9]:
# display_all(df)
# display(df)

In [12]:
year_start=2000
year_end=2021
dfdict={}
for year in range(year_start,year_end+1):
    df = mygus.read_xls_year(year)
    df = mygus.format_df(df,year)
    print(year)
    dfdict[year]=df



2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [28]:
# dfdict[2021]
pd.concat(list(dfdict.values()), ignore_index=True)

,Wiek zmarłych w latach,NUTS,Podregiony,T01,T02,T03,T04,T05,T06,T07,...,T44,T45,T46,T47,T48,T49,T50,T51,T52,2021
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9309,9739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
1,Ogółem,PL,Polska,7913,8292,8970,9380,9474,8591,7841,...,1780,1920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
2,Ogółem,PL2,Makroregion Południowy,1629,1695,1805,1792,1754,1597,1520,...,724,779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
3,Ogółem,PL21,Małopolskie,616,636,676,675,665,649,548,...,159,176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
4,Ogółem,PL213,Miasto Kraków,139,162,156,167,162,154,135,...,146,158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,90 i więcej,PL922,Ciechanowski,8,3,3,5,4,4,11,...,8,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
1957,90 i więcej,PL923,Płocki,10,7,2,9,7,10,7,...,10,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
1958,90 i więcej,PL924,Ostrołęcki,8,6,10,7,11,8,5,...,27,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
1959,90 i więcej,PL925,Siedlecki,11,10,5,14,8,4,9,...,13,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
